## 🔄 Conversão Automática de Notebooks Jupyter para Scripts Python  
### Automação da padronização e limpeza de notebooks em pipelines de produção

### 🧭 Configuração Inicial do Ambiente  
##  
Importa módulos essenciais do sistema (`os`, `subprocess`, `re`, `datetime`) para manipulação de arquivos, execução de comandos externos e controle de tempo.  
Essas bibliotecas fornecem as bases para a automação do processo de conversão dos notebooks `.ipynb` em scripts `.py`.

In [2]:
import os
from datetime import datetime

import nbformat
from nbconvert.exporters import ScriptExporter

### 🔁 Função `converter_notebook_para_py()`  
##  
Converte um arquivo `.ipynb` (Jupyter Notebook) em um script Python `.py` no mesmo diretório.  
O processo utiliza o comando interno do Jupyter:  
```bash
jupyter nbconvert --to script

In [3]:
def converter_notebook_para_py(caminho_notebook: str) -> bool:
    """
    Converte um único arquivo .ipynb em .py usando nbconvert.
    Retorna True em caso de sucesso, False em caso de erro.
    """
    try:
        dir_base, nome_arquivo = os.path.split(caminho_notebook)
        nome_base, _ = os.path.splitext(nome_arquivo)
        caminho_py = os.path.join(dir_base, f"{nome_base}.py")

        with open(caminho_notebook, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        exporter = ScriptExporter()
        script, _ = exporter.from_notebook_node(nb)

        with open(caminho_py, "w", encoding="utf-8") as f:
            f.write(script)

        print(f"✅ {datetime.now():%H:%M:%S} | Convertido: {caminho_notebook}")
        return True

    except Exception as e:
        print(f"❌ {datetime.now():%H:%M:%S} | Erro ao converter {caminho_notebook}: {e}")
        return False

### 📂 Função `converter_notebooks_em_pastas()`  
##  
Percorre uma ou mais pastas e converte todos os arquivos `.ipynb` encontrados (inclusive em subpastas, se habilitado).  
A função exibe um relatório completo ao final da execução, incluindo:  

- Quantidade total de notebooks encontrados  
- Quantos foram convertidos com sucesso  
- Quantos apresentaram erro  
- Duração total da operação  

Esse processo garante uma conversão automatizada em lote, ideal para pipelines de produção e repositórios versionados.

In [4]:
def converter_notebooks_em_pastas(lista_pastas, incluir_subpastas: bool = True) -> None:
    """
    Converte todos os .ipynb em uma ou mais pastas (opcionalmente incluindo subpastas)
    e mostra um resumo ao final.
    """
    total_encontrados = 0
    total_convertidos = 0
    total_erros = 0
    inicio = datetime.now()

    print(f"\n🚀 Início: {inicio:%Y-%m-%d %H:%M:%S}")
    print("------------------------------------------------")

    for pasta in lista_pastas:
        if not os.path.exists(pasta):
            print(f"[ERRO] Pasta não encontrada: {pasta}")
            continue

        print(f"\n📂 Procurando notebooks em: {pasta}")

        iterador = os.walk(pasta) if incluir_subpastas else [(pasta, [], os.listdir(pasta))]
        for raiz, _, arquivos in iterador:
            for arquivo in arquivos:
                if arquivo.endswith(".ipynb"):
                    total_encontrados += 1
                    caminho_notebook = os.path.join(raiz, arquivo)
                    if converter_notebook_para_py(caminho_notebook):
                        total_convertidos += 1
                    else:
                        total_erros += 1

    fim = datetime.now()
    duracao = (fim - inicio).total_seconds()

    print("\n------------------------------------------------")
    print("📊 Resumo:")
    print(f"  🧾 Total encontrado : {total_encontrados}")
    print(f"  ✅ Convertidos       : {total_convertidos}")
    print(f"  ❌ Erros             : {total_erros}")
    print(f"  ⏱️ Duração total     : {duracao:.1f} segundos")
    print(f"🏁 Término: {fim:%Y-%m-%d %H:%M:%S}")
    print("------------------------------------------------\n")

### 🚀 Execução Principal (`__main__`)  
##  
Define as pastas que contêm os notebooks a serem processados e chama a função de conversão em lote.  
No exemplo configurado, são processadas as seguintes pastas:  

- `a_transform_actual`  
- `a_transform_target`  
- `b_analytics`  
- `c_send_message`  

Ao ser executado diretamente, o script realiza a conversão de todos os notebooks dessas pastas em scripts Python prontos para execução automatizada.

In [5]:
# ===== EXEMPLO DE USO =====
if __name__ == "__main__":
    pastas = [
        '../export',
    ]
    converter_notebooks_em_pastas(pastas)


🚀 Início: 2025-12-03 11:23:53
------------------------------------------------

📂 Procurando notebooks em: ../export
✅ 11:23:53 | Convertido: ../export\ETL.ipynb
✅ 11:23:54 | Convertido: ../export\load_Supabase.ipynb

------------------------------------------------
📊 Resumo:
  🧾 Total encontrado : 2
  ✅ Convertidos       : 2
  ❌ Erros             : 0
  ⏱️ Duração total     : 0.7 segundos
🏁 Término: 2025-12-03 11:23:54
------------------------------------------------



### 📘 Resumo Técnico do Script  
##  
**Objetivo:**  
Automatizar a conversão e limpeza de notebooks Jupyter (`.ipynb`) em scripts Python (`.py`), garantindo conformidade estrutural e compatibilidade com o ambiente de execução do projeto.  

**Principais Transformações:**  
- Conversão automática via `nbconvert`.  
- Limpeza inteligente do código gerado, removendo redundâncias e descomentando trechos essenciais.  
- Execução em lote com relatório de progresso e logs claros.  

**Saída:**  
Scripts `.py` limpos e padronizados, prontos para execução autônoma dentro do ambiente do projeto, mantendo compatibilidade e rastreabilidade completas.